**1. Import dataset**

In [1]:
import pandas as pd

df = pd.read_parquet('./data/data_final.parquet')

print(df.head(5))

                                               title  \
0  Is Physics Sick? [In Praise of Classical Physics]   
1    Modern Mathematical Physics: what it should be?   
2                                Topology in Physics   
3       Contents of Physics Related E-Print Archives   
4        Fundamental Dilemmas in Theoretical Physics   

                                             authors  \
0                                     Hisham Ghassib   
1                                     Ludwig Faddeev   
2                                          R. Jackiw   
3  E. R. Prakasan, Anil Kumar, Anil Sagar, Lalit ...   
4                                     Hisham Ghassib   

                                             summary             published  \
0  In this paper, it is argued that theoretical p...  2012-09-04T10:32:56Z   
1  Personal view of author on goals and content o...  2000-02-08T13:13:00Z   
2  The phenomenon of quantum number fractionaliza...  2005-03-15T16:00:59Z   
3  The frontie

**2. Preprocessing data**

In [2]:
import re
import string
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords

# Download necessary NLTK data (run once)
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the tokenizer, lemmatizer, stemmer
wpt = WordPunctTokenizer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Create a stopwords set
stop_words = set(stopwords.words('english'))

def normalized_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'<.*?>+', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', ' ', text)
    tokens = wpt.tokenize(text)
    
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    lemma_stem_tokens = [stemmer.stem(lemmatizer.lemmatize(token)) for token in filtered_tokens]
    
    cleaned_text = ' '.join(lemma_stem_tokens)
    return cleaned_text


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tranminhanh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tranminhanh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df['cleaned'] = df['summary'].apply(lambda x: normalized_text(x))

**3. Training model**

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, hamming_loss, jaccard_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier

# Convert comma-separated target strings into lists
df["target_list"] = df["target"].apply(lambda x: x.split(",") if isinstance(x, str) else [])

# Apply binary encoding to target column
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['target_list'])

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)  # Limit features to 5000 for efficiency
X_tfidf = tfidf.fit_transform(df["cleaned"])

# Apply Truncated SVD (Dimensionality Reduction) to 50 components
svd = TruncatedSVD(n_components=50)  # Reduce TF-IDF features to 50 components
X_svd = svd.fit_transform(X_tfidf)  # Now X_svd has shape (n, 50)

# Step 4: Split data into training and temp (validation + test)
X_train, X_temp, Y_train, Y_temp = train_test_split(X_svd, y, test_size=0.2, random_state=42)

# Further split temp into validation and test sets (50% each)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

# Initialize XGBoost Classifier with binary logistic objective
xgb_model = XGBClassifier(objective="binary:logistic", eval_metric="logloss", use_label_encoder=False)

# Wrap with MultiOutputClassifier for multi-label classification
model = MultiOutputClassifier(xgb_model)

# Hyperparameter tuning with GridSearchCV (learning rate < 0.05)
param_grid = {
    'estimator__max_depth': [3, 6, 10],
    'estimator__learning_rate': [0.01, 0.03, 0.05, 0.001, 0.005],  # Learning rate values under 0.05
    'estimator__n_estimators': [100, 200, 250, 300]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)

# Apply class weights to the XGBoost model
xgb_model = XGBClassifier(objective="binary:logistic", eval_metric="logloss", use_label_encoder=False, scale_pos_weight=1)

# Wrap with MultiOutputClassifier for multi-label classification
model = MultiOutputClassifier(xgb_model)

# Fit the model with the training data
grid_search.fit(X_train, Y_train)

# Best hyperparameters found by GridSearchCV
print("Best Hyperparameters: ")
print(grid_search.best_params_)

# Predictions on test set
y_test_pred = grid_search.best_estimator_.predict(X_test)

# Calculate test metrics
test_f1 = f1_score(Y_test, y_test_pred, average='macro')
test_precision = precision_score(Y_test, y_test_pred, average='macro')
test_recall = recall_score(Y_test, y_test_pred, average='macro')
test_hamming = hamming_loss(Y_test, y_test_pred)
test_jaccard = jaccard_score(Y_test, y_test_pred, average='macro')

# Print the test metrics
print("Test Classification Report:")
print(classification_report(Y_test, y_test_pred, target_names=mlb.classes_))
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test Hamming Loss: {test_hamming:.4f}")
print(f"Test Jaccard Score: {test_jaccard:.4f}")


Fitting 3 folds for each of 60 candidates, totalling 180 fits


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:01:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:01:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:01:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:01:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

[CV] END estimator__learning_rate=0.01, estimator__max_depth=3, estimator__n_estimators=100; total time=   9.6s
[CV] END estimator__learning_rate=0.01, estimator__max_depth=3, estimator__n_estimators=100; total time=   9.6s
[CV] END estimator__learning_rate=0.01, estimator__max_depth=3, estimator__n_estimators=100; total time=   9.6s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:01:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:01:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:01:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:01:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

[CV] END estimator__learning_rate=0.01, estimator__max_depth=3, estimator__n_estimators=200; total time=  17.6s
[CV] END estimator__learning_rate=0.01, estimator__max_depth=3, estimator__n_estimators=200; total time=  17.7s


...

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:16:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:16:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END estimator__learning_rate=0.005, estimator__max_depth=10, estimator__n_estimators=300; total time= 2.5min
[CV] END estimator__learning_rate=0.005, estimator__max_depth=10, estimator__n_estimators=300; total time= 2.4min
[CV] END estimator__learning_rate=0.005, estimator__max_depth=10, estimator__n_estimators=300; total time= 2.3min


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:17:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:17:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:17:15] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:17:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

Best Hyperparameters: 
{'estimator__learning_rate': 0.05, 'estimator__max_depth': 10, 'estimator__n_estimators': 300}
Test Classification Report:
              precision    recall  f1-score   support

         bio       0.82      0.54      0.65      1074
          cs       0.87      0.85      0.86      5758
   econ-qfin       0.81      0.73      0.77      1240
        eess       0.75      0.43      0.55      1007
  math-stats       0.85      0.76      0.80      5432
      physic       0.90      0.80      0.85      5550

   micro avg       0.87      0.77      0.81     20061
   macro avg       0.83      0.69      0.75     20061
weighted avg       0.86      0.77      0.81     20061
 samples avg       0.87      0.82      0.82     20061

Test F1 Score: 0.7472
Test Precision: 0.8337
Test Recall: 0.6865
Test Hamming Loss: 0.0782
Test Jaccard Score: 0.6088


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**4. Backtesting**

In [6]:
import numpy as np

# Step 1: Generate Synthetic Data Similar to Your Dataset
new_data_samples = [
    "Quantum entanglement and Bell's theorem challenge classical interpretations of physics.",
    "Statistical methods in probability theory play a fundamental role in mathematical modeling.",
    "New deep learning architectures are transforming natural language processing applications.",
    "Cryptography relies on number theory and complex mathematical algorithms.",
    "Bayesian inference is widely used in statistical decision-making and machine learning."
]

# Step 2: Preprocess New Data (Apply Same Vectorization)
new_data_vectorized = tfidf.transform(new_data_samples)  # Replace 'tfidf' with your fitted vectorizer

# Step 3: Apply the same dimensionality reduction (SVD) as was done during training
new_data_svd = svd.transform(new_data_vectorized)  # Apply the same SVD transformation

# Step 4: Predict Using the Best Model from GridSearchCV
new_predictions = grid_search.best_estimator_.predict(new_data_svd)

# Step 5: Convert Binary Predictions Back to Labels
predicted_labels = mlb.inverse_transform(new_predictions)  # Convert binary predictions to labels

# Step 6: Display Predictions
for i, text in enumerate(new_data_samples):
    print(f"\n🔹 **Generated Test Data Point {i+1}**: {text}")
    print(f"📌 **Predicted Categories**: {predicted_labels[i]}")



🔹 **Generated Test Data Point 1**: Quantum entanglement and Bell's theorem challenge classical interpretations of physics.
📌 **Predicted Categories**: ('math-stats', 'physic')

🔹 **Generated Test Data Point 2**: Statistical methods in probability theory play a fundamental role in mathematical modeling.
📌 **Predicted Categories**: ()

🔹 **Generated Test Data Point 3**: New deep learning architectures are transforming natural language processing applications.
📌 **Predicted Categories**: ('cs', 'math-stats')

🔹 **Generated Test Data Point 4**: Cryptography relies on number theory and complex mathematical algorithms.
📌 **Predicted Categories**: ('math-stats',)

🔹 **Generated Test Data Point 5**: Bayesian inference is widely used in statistical decision-making and machine learning.
📌 **Predicted Categories**: ('math-stats',)
